# Testing dataset

## Importing library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler

## Loading datasets

In [2]:
df=pd.read_csv("E:/Kaggel compitiion/House Prices Advanced Regression Techniques/test.csv")

## Missing Values

In [3]:
# Missing values in categorical features
missing_features=[i for i in df.columns if df[i].isnull().sum()>1 and df[i].dtype=='O']
for i in missing_features:
    print(i,np.round(df[i].isnull().mean(),4))


MSZoning 0.0027
Alley 0.9267
Utilities 0.0014
MasVnrType 0.011
BsmtQual 0.0302
BsmtCond 0.0308
BsmtExposure 0.0302
BsmtFinType1 0.0288
BsmtFinType2 0.0288
Functional 0.0014
FireplaceQu 0.5003
GarageType 0.0521
GarageFinish 0.0535
GarageQual 0.0535
GarageCond 0.0535
PoolQC 0.9979
Fence 0.8012
MiscFeature 0.965


In [4]:
# Replacing Missing value with new label
def fillNa(df,missing_features):
    data=df.copy()
    data[missing_features]=data[missing_features].fillna("Missing")
    return data
df=fillNa(df,missing_features)
print(df[missing_features].isnull().sum())

MSZoning        0
Alley           0
Utilities       0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Functional      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64


In [5]:
#Missing values in numerical variable
missing_features1=[i for i in df.columns if df[i].isnull().sum()>1 and df[i].dtype!='O']
for i in missing_features1:
    print(i,np.round(df[i].isnull().mean(),4))

LotFrontage 0.1556
MasVnrArea 0.0103
BsmtFullBath 0.0014
BsmtHalfBath 0.0014
GarageYrBlt 0.0535


In [6]:
# Now we will replace missing value in numerical by median because some values are very large
# And creating new feature if missing then 1 else 0
for i in missing_features1:
    med=df[i].median()
    #Creatiing new feature 
    df[i+"Nan"]=np.where(df[i].isnull(),1,0)
    #Filling missing value with median
    df[i].fillna(med,inplace=True)
    
df[missing_features1].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
dtype: int64

## Converting year to how many year

In [7]:
yr_features=["GarageYrBlt","YearRemodAdd","YearBuilt"]
for i in yr_features:
        df[i]=df['YrSold']-df[i]

## Appling gaussian distribution

In [8]:
# Appling to them which features do not have 0
# numerical_features=[i for i in df.columns if df[i].dtype !="O" if 0 not in df[i].unique()]

numerical_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']
for i in numerical_features:
    df[i]=np.log(df[i])

## Handeling Rare categorical Features

In [9]:
#In this we skip that categorical featres which has less than 1%  

categorical_features=[i for i in df.columns if df[i].dtype =="O" ]
for i in categorical_features:
    temp=df.groupby(i)["Id"].count()/len(df)
    temp_df=temp[temp>0.01].index
    df[i]=np.where(df[i].isin(temp_df),df[i],"Rare_var")

## Label-Encoding

In [10]:
#Changing Categorical features
for feature in categorical_features:
    labels_ordered=df.groupby([feature])['Id'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

## Scaling

In [11]:
#Appling scaling either min max scalong or stanfard scaling
scale_features=[i for i in df.columns if i not in ['Id']]
scale=MinMaxScaler()
scale.fit(df[scale_features])

MinMaxScaler(copy=True, feature_range=(0, 1))

## Saving

In [ ]:
#Doing this concat beacuse the scale.transorfm gives array we have to change into dataframe
data = pd.concat([df[['Id']].reset_index(drop=True),
                    pd.DataFrame(scale.transform(df[scale_features]), columns=scale_features)],
                    axis=1)
data.to_csv(r'E:/Kaggel compitiion/house-prices-advanced-regression-techniques/After analizing/test.csv',index=False)
